In [ ]:
from langchain.embeddings.openai import OpenAIEmbeddings 
from langchain.text_splitter import CharacterTextSplitter 
from langchain.vectorstores import ElasticVectorSearch, Pinecone, Weaviate
from langchain import OpenAI, ConversationChain

from langchain.document_loaders import UnstructuredPDFLoader, OnlinePDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter

# Import PyPDF2
import PyPDF2
import numpy as np 
import torch
device = torch.cuda.current_device() if torch.cuda.is_available() else None

In [ ]:
from transformers import AutoTokenizer, AutoModelForTokenClassification
from transformers import pipeline

model_id = "dslim/bert-base-NER"

# load the tokenizer from huggingface
tokenizer = AutoTokenizer.from_pretrained(
    model_id
)
# load the NER model from huggingface
model = AutoModelForTokenClassification.from_pretrained(
    model_id
)
# load the tokenizer and model into a NER pipeline
nlp = pipeline(
    "ner",
    model=model,
    tokenizer=tokenizer,
    aggregation_strategy="max",
    device=device
)

In [19]:
import os 
# Load openai api key 
openai_api_key = os.environ.get('OPENAI_API_KEY')

# Load Pinecone API key 
pinecone_api_key = os.environ.get('PINECONE_API_KEY')

# Load pinecode api env 
pinecone_api_env = os.environ.get('PINECONE_API_ENV')

In [20]:
llm = OpenAI(temperature=0.9)
from langchain.prompts import PromptTemplate
prompt = PromptTemplate(
    input_variables=["food"],
    template="What are 5 vacation destinations for someone who likes to eat {food}?",
)

In [21]:
print(prompt.format(food="dessert"))

What are 5 vacation destinations for someone who likes to eat dessert?


In [22]:
print(llm(prompt.format(food="dessert")))



1. Paris, France - Home of the famous patisseries such as Laduree with its signature macarons and Pierre Herme's decadent cakes.

2. Tokyo, Japan - With an ever-growing range of interesting and complex desserts from fluffy souffles to mochi ice cream, Tokyo is a great destination for dessert lovers.

3. New York City, United States - A seemingly never-ending array of dessert spots, from the classic cronuts to inventive ice cream flavors, NYC has something for everyone.

4. Milan, Italy - An embarrassment of riches when it comes to desserts, Milan offers traditional favorites like gelato and Tiramisu, as well as modern takes on traditional cakes and pastries.

5. Vienna, Austria - Famous for its Sachertorte and apple strudel, Vienna has a deep history of decadent desserts to explore.


In [23]:
from langchain.prompts import PromptTemplate
from langchain.llms import OpenAI
from langchain.chains import LLMChain

llm = OpenAI(temperature=0.9)

prompt = PromptTemplate(
    input_variables=["food"],
    template="What are 5 vacation destinations for someone who likes to eat {food}?",
)

chain = LLMChain(llm=llm, prompt=prompt)
print(chain.run("fruit"))



1. Costa Rica - known for its delicious tropical fruits such as papayas, pineapples, and bananas.

2. India - expore the diverse culinary traditions of India and you'll find an abundance of sweet, juicy tropical fruits like Mangos and Guavas.

3. Brazil - taste all the exotic fruits such as Cupuaçu, Cherimoyas, and Tomates de Arvobia.

4. Thailand - enjoy an array of amazing fruits like mangosteen, Thai dragonfruit, and rambutan.

5. Hawaii - savor the sweet taste of fresh pineapples, starfruit, and lychees.


In [24]:
!pip install google-search-results


[notice] A new release of pip is available: 23.0.1 -> 23.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [26]:
from langchain.agents import load_tools
from langchain.agents import initialize_agent
from langchain.agents import AgentType
from langchain.chat_models import ChatOpenAI
from langchain.llms import OpenAI

In [27]:
# Load in some tools to use

# os.environ["SERPAPI_API_KEY"] = "..."

tools = load_tools(["serpapi", "llm-math"], llm=llm)

In [28]:
# Finally, let's initialize an agent with:
# 1. The tools
# 2. The language model
# 3. The type of agent we want to use.

agent = initialize_agent(tools, llm, agent="zero-shot-react-description", verbose=True)

In [29]:
# Now let's test it out!
agent.run("Who is the current leader of Ycombinator? What are the most successful companies on the portfolio?")



> Entering new AgentExecutor chain...
 Y Combinator is a venture capital firm, so I need to research their leaders and successful companies.
Action: Search
Action Input: Ycombinator leader, successful Ycombinator companies
Observation: No good search result found
Thought: I should look for a website that lists Y Combinator information
Action: Search
Action Input: Ycombinator website
Observation: Y Combinator created a new model for funding early stage startups. ... Twice a year we invest $500,000 per company in a large number of startups. We work ...
Thought: The website looks like it has the information I need
Action: Search
Action Input: Ycombinator leader, successful Ycombinator companies website
Observation: No good search result found
Thought: Maybe I can find the website on a venture capital resource site
Action: Search
Action Input: venture capital website Ycombinator
Observation: Y Combinator created a new model for funding early stage startups. Twice a year we invest $500,00

'The current leader of Y Combinator is Garry Tan, and the most successful companies in their portfolio include Snapdocs and Airbnb.'

In [ ]:
from langchain import OpenAI, ConversationChain

In [ ]:
import pinecone 
index = pinecone.Index("mergers-and-acqs")
index_description = pinecone.describe_index("mergers-and-acqs")
index_stats_response = index.describe_index_stats()

In [ ]:
index_stats_response

In [ ]:
# Extract text from the PDF
def extract_text_from_pdf(file_path):
    with open(file_path, 'rb') as file:
        reader = PyPDF2.PdfFileReader(file)
        text = ''
        for page_num in range(reader.numPages):
            text += reader.getPage(page_num).extractText()
    return text

def get_all_pdf_files_in_directory(directory):
    return [os.path.join(directory, f) for f in os.listdir(directory) if f.endswith('.pdf')]

In [ ]:
import pandas as pd
import numpy as np
import time
import re
from wordcloud import WordCloud, STOPWORDS, ImageColorGenerator
import matplotlib.pyplot as plt
import sqlite3

pd.set_option('display.max_colwidth', 200)

In [ ]:
import os
import PyPDF2
import pinecone
from sentence_transformers import SentenceTransformer

In [ ]:
data_directory = "data/"
pdf_files = get_all_pdf_files_in_directory(data_directory)

In [ ]:
pdf_files 

In [ ]:
import faiss

In [ ]:
bnr_report_reader = UnstructuredPDFLoader("data/Annual_Report_2021_22_Web_English_Versio.pdf")
bnr_report_reader_data = bnr_report_reader.load()

embeddings = OpenAIEmbeddings(openai_api_key=openai_api_key)
llm = OpenAI(temperature=0, openai_api_key=openai_api_key)
conversation = ConversationChain(llm=llm, verbose=True)

In [ ]:
bnr_report_reader_data

In [ ]:
index = faiss.IndexFlatL2(bnr_report_reader_data)

In [ ]:
from langchain.document_loaders import PyPDFLoader

loader = PyPDFLoader("data/Annual_Report_2021_22_Web_English_Versio.pdf")
pages = loader.load_and_split()

In [ ]:
# Pick a name for the new index
simple_index_name = 'stocks-trends'

In [ ]:
# Check whether the index with the same name already exists
if simple_index_name in pinecone.list_indexes():
    pinecone.delete_index(simple_index_name)

In [ ]:
# Create a new index
pinecone.create_index(name=simple_index_name, dimension=128)

In [ ]:
# Define nlp
import spacy
nlp = spacy.load("en_core_web_sm")

def extract_named_entities(text_batch):
    # extract named entities using the NER pipeline
    extracted_batch = nlp(text_batch)
    entities = []
    # loop through the results and only select the entity names
    for text in extracted_batch:
        ne = [entity["word"] for entity in text]
        entities.append(ne)
    return entities

In [ ]:
pinecone.list_indexes()

In [ ]:
pinecone.describe_index("pinecone-index")

In [ ]:
pinecone.create_index("example-index", dimension=128, metric="euclidean", pods=4, pod_type="s1.x1")

In [ ]:
pinecone.create_index("example-index", dimension=128, source_collection="example-collection")

In [ ]:
pinecone.configure_index("my_index", pod_type="s1.x2")

In [ ]:
pinecone.describe_index("example-index")

In [ ]:
pinecone.configure_index("example-index", replicas=4)

In [ ]:
from pprint import pprint
# Define retriever
retriever = SentenceTransformer('paraphrase-distilroberta-base-v1')

# Define index
index = pinecone.Index()

def search_pinecone(query):
    # extract named entities from the query
    ne = extract_named_entities([query])[0]
    # create embeddings for the query
    xq = retriever.encode(query).tolist()
    # query the pinecone index while applying named entity filter
    xc = index.query(xq, top_k=10, include_metadata=True, filter={"named_entities": {"$in": ne}})
    # extract article titles from the search result
    r = [x["metadata"]["title"] for x in xc["matches"]]
    return pprint({"Extracted Named Entities": ne, "Result": r})

In [ ]:
from langchain.vectorstores import FAISS
from langchain.embeddings.openai import OpenAIEmbeddings

faiss_index = FAISS.from_documents(pages, OpenAIEmbeddings(openai_api_key=openai_api_key))
docs = faiss_index.similarity_search("What are the top economic challenges facing Rwanda?", k=2)
for doc in docs:
    print(str(doc.metadata["page"]) + ":", doc.page_content)

In [ ]:
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=0)
texts = text_splitter.split_documents(pages)
print (f'Now you have {len(texts)} documents')

In [ ]:
import pinecone

pinecone.init(api_key=pinecone_api_key)

pinecone.create_index("national_bank_index", dimension=1024)

In [ ]:
pinecone.init(api_key="YOUR_API_KEY", environment="YOUR_ENVIRONMENT")
index = pinecone.Index("example-index")

index_stats_response = index.describe_index_stats()

In [ ]:
import chromadb
# setup Chroma in-memory, for easy prototyping. Can add persistence easily!
client = chromadb.Client()

# create a new index 
index = client.create_index("bnr_index")

# add documents to the index
for text in texts:
    index.add_document(text)

# search for similar documents  
results = index.search("What are the top economic challenges facing Rwanda?", k=2)
for result in results:
    print(result)

In [ ]:
from langchain.vectorstores import Pinecone
from langchain.embeddings.openai import OpenAIEmbeddings

In [ ]:
import pinecone

pinecone.init(api_key=pinecone_api_key)
index = pinecone.Index("example-index")

In [ ]:
metadata_config = {'indexed': ['color']}

In [ ]:
# Use Pinecone to implement a vector store 
pinecone_index = Pinecone(index_name="example-index", metadata_config=metadata_config)

# Use OpenAI to implement an embedding model
openai_embeddings = OpenAIEmbeddings(openai_api_key=openai_api_key)

# Use Weaviate to implement a vector store  

weaviate_index = Weaviate("http://localhost:8080", "bnr_index")

# Use FAISS to implement a vector store
faiss_index = FAISS.from_documents(pages, OpenAIEmbeddings(openai_api_key=openai_api_key))

In [ ]:

# Use approximate nearest neighbor search to find similar documents
docs = faiss_index.similarity_search("What are the top economic challenges facing Rwanda?", k=2)
for doc in docs:
    print(str(doc.metadata["page"]) + ":", doc.page_content)